In [1]:
import scipy
from scipy.io import wavfile
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal
import osascript
from gtts import gTTS 
import os 
import pyaudio
import wave
import keyboard  # using module keyboard
import soundfile as sf
import math
import pyloudnorm as pyln
from sys import byteorder
from array import array
from struct import pack
import librosa
from scipy.signal import butter, sosfiltfilt
import python_speech_features
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from ipynb.fs.full.Pitch_vector import get_pitch_stats
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 


Using TensorFlow backend.


In [2]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], btype='band', analog=False, output='sos')
    return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfiltfilt(sos, data)
    return y

In [3]:
def extract_data(file_location):
    fs, data = wavfile.read(file_location)
    number_of_samples = data.shape[0]
    meta_data = open(r"LDC2002S28-txt.txt")
    meta_data = pd.read_csv("LDC2002S28-txt.txt", sep="A:", header=None, engine='python')
    meta_data.columns = ["sound limits","description"]
    
    #dual channel to one channel
    data = np.average(data, axis = 1)
    #remove noise
    data = butter_bandpass_filter(data, BANDPASS_FREQ[0], BANDPASS_FREQ[1], fs)
    
    # removee extra data pointts
    meta_data = meta_data[meta_data.description != ' [MISC]']
    meta_data = meta_data[~meta_data['description'].astype(str).str.startswith(' (')]
    meta_data = meta_data[~meta_data['description'].astype(str).str.startswith(' Emotion category elation')]
    meta_data = meta_data[~meta_data['description'].astype(str).str.startswith('  [MISC]')]

    # description and time limits 
    voice_time_limits = meta_data["sound limits"]
    voice_time_limits = [i.split(" ")[0:2] for i in voice_time_limits]
    voice_time_limits = np.array(voice_time_limits)
    voice_time_limits = voice_time_limits.astype(np.float)
    description = meta_data["description"]
    description = [i.split(",")[0].strip() for i in description]

    #divide the dataa set
    divided_data = []
    for i in voice_time_limits:
        startingpoint = int(i[0]*fs)
        endingpoint = int(i[1]*fs)
        divided_data.append(data[startingpoint:endingpoint])
    np_data = np.asarray(divided_data)
    return np_data, description, len(divided_data), fs

In [4]:
def MFCC_algorithm(np_data):
        # MFCC function taking the first thirteen coef
    MFCC2 = []
    for i in np_data:
        i = np.asarray(i)
        MFCC2.append(python_speech_features.base.mfcc(i, samplerate=fs, 
                                     winlen=0.025, winstep=0.01, numcep=13, 
                                     nfilt=26, nfft=552))
        
    # gather information from the MFCC (feature extraction)
    MFCC3 = []
    cache = {}
    for data_point in MFCC2:
        for time_segment in data_point:
            if (data_point[0] == time_segment).all():
                for i in range(13):
                    cache[i] = [time_segment[i]]
            else:
                for i in range(13):
                    cache[i] = np.concatenate((cache[i], [time_segment[i]]))
        cached_variables = []
        cache_grad = []
        for i in range(13):
            cache_grad.append(np.gradient(cache[i]))
            cached_variables.append([np.mean(cache[i]), np.median(cache[i]), np.var(cache[i]), 
                               np.min(cache[i]), np.max(cache[i]), 
                                     np.mean(cache_grad[i]), np.var(cache_grad[i])])
        MFCC3.append(np.hstack(np.hstack(cached_variables)))
    return MFCC3
    

In [5]:

def emotion_extraction(description, number_examples):
    nu_emotion = 15
    y = np.zeros(shape=(nu_emotion, number_examples))
    counter = 0
    for i in description:
        X0 = np.zeros((number_examples,1))
        if i == 'neutral':
            y[0][counter] = 1
        elif i == 'disgust':
             y[1][counter] = 1
        elif i == 'panic':
             y[2][counter] = 1
        elif i == 'anxiety':
             y[3][counter] = 1
        elif i == 'hot anger':
             y[4][counter] = 1
        elif i == 'cold anger':
             y[5][counter] = 1
        elif i == 'despair':
             y[6][counter] = 1
        elif i == 'sadness':
             y[7][counter] = 1
        elif i == 'elation':
             y[8][counter] = 1
        elif i == 'happy':
             y[9][counter] = 1
        elif i == 'interest':
             y[10][counter] = 1
        elif i == 'boredom':
             y[11][counter] = 1
        elif i == 'shame':
             y[12][counter] = 1
        elif i == 'pride':
             y[13][counter] = 1
        elif i == 'contempt':
             y[14][counter] = 1
        else:
            print(i)
            break
        counter +=1
    y = np.transpose(y)
    return y

In [6]:
def MLPalgorithm(X_train, X_test, y_train, y_test):
    mlp = MLPClassifier(hidden_layer_sizes=(20,30,15),max_iter=15000)
    mlp.fit(X_train,y_train)
    predictions = mlp.predict(X_train)
    print(classification_report(y_train,predictions))
    predictions = mlp.predict(X_test)
    print(classification_report(y_test,predictions))

In [7]:
def get_data(filepath):
    data = np.load(filepath, allow_pickle=True)
    fs = RATE
    return data, fs